In [1]:
import os
os.chdir('..')

In [2]:
import tensorflow.keras as keras
import tensorflow as tf
from lib.utils import load_train, load_validation
from lib.preprocessing import OneHotLabels, ResizeImage, FloatifyImage

/home/matt/tools/miniconda3/envs/kaggle-hp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/matt/tools/miniconda3/envs/kaggle-hp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, Flatten
from tensorflow.keras.layers import MaxPooling2D
inputs = Input(shape=(264, 264, 4))
x = Conv2D(64, (3, 3), activation=tf.nn.relu)(inputs)
x = MaxPooling2D((2, 2))(x)

print(x.get_shape())
x = Conv2D(64, (3, 3), activation=tf.nn.relu)(x)
x = MaxPooling2D((2, 2))(x)
print(x.get_shape())

x = Conv2D(64, (3, 3), activation=tf.nn.relu)(x)
x = MaxPooling2D((2, 2))(x)
print(x.get_shape())

x = Flatten()(x)
print(x.get_shape())


x = Dense(512, activation=tf.nn.relu)(x)
print(x.get_shape(), x.name)

outputs = Dense(28, activation=tf.nn.sigmoid)(x)
print(outputs.get_shape())

model = keras.Model(inputs=inputs, outputs=outputs)

optimizer = tf.train.AdamOptimizer(0.001)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

(?, 131, 131, 64)
(?, 64, 64, 64)
(?, 31, 31, 64)
(?, 61504)
(?, 512) dense/Relu:0
(?, 28)


In [4]:
estimator = tf.keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpyblqbyrg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1d748400f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
def _one_hot_labels(ids, image, labels):
    ohe_labels = tf.reduce_sum(tf.one_hot(labels, 28), axis=0)
    return ids, image, ohe_labels

def _preprocess_img(ids, image, labels):
    resized_img = tf.image.resize_images(image, (264, 264))
    floated_img = resized_img / 255
    return floated_img, labels

In [5]:
def small_input_fn(record_dataset):
    small = record_dataset.take(200)
    small = small.map(FloatifyImage())
    small = small.map(OneHotLabels(28))
    small = small.map(ResizeImage((264, 264)))
    small = small.batch(16)
    small = small.repeat(10)
    return small

In [6]:
estimator.train(input_fn=lambda:small_input_fn(load_train()))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpyblqbyrg/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmpyblqbyrg/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: conv2d/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_2/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense/bias; prev_var_nam

KeyboardInterrupt: 

In [8]:
img_val = sess.run(img)
model.predict(img_val)

ResourceExhaustedError: OOM when allocating tensor with shape[14565600,120] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node dense/kernel/Initializer/random_uniform/RandomUniform}} = RandomUniform[T=DT_INT32, _class=["loc:@dense/kernel"], dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense/kernel/Initializer/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense/kernel/Initializer/random_uniform/RandomUniform', defined at:
  File "/home/matt/tools/miniconda3/envs/kaggle-hp/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/matt/tools/miniconda3/envs/kaggle-hp/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/matt/.local/lib/python3.6/site-packages/ipykernel_launcher.py",

In [9]:
import numpy as np
test = np.random.rand(4, 256, 256, 4)

In [12]:
model.predict(test)

array([[0.543608  , 0.6057506 , 0.54024845, 0.5211303 , 0.44120902,
        0.4898803 , 0.4889393 , 0.4997212 , 0.38905522, 0.48498416,
        0.46905607, 0.38942787, 0.516596  , 0.501891  , 0.42107642,
        0.5370104 , 0.63855404, 0.3964052 , 0.4741492 , 0.4788913 ,
        0.4655195 , 0.55945283, 0.50151575, 0.48820716, 0.5200031 ,
        0.57775855, 0.461616  , 0.50629985],
       [0.5472436 , 0.5938977 , 0.5435564 , 0.5312742 , 0.4419791 ,
        0.487079  , 0.49413526, 0.505871  , 0.39326125, 0.47470388,
        0.4633972 , 0.38063833, 0.5016582 , 0.5002924 , 0.41086775,
        0.532996  , 0.64571565, 0.4114451 , 0.48374486, 0.4824356 ,
        0.45935184, 0.5674189 , 0.5016082 , 0.48020396, 0.4989728 ,
        0.5690943 , 0.4638389 , 0.51010454],
       [0.54599255, 0.5955546 , 0.5425559 , 0.52257824, 0.4548226 ,
        0.47945303, 0.48746377, 0.4988875 , 0.3907123 , 0.47804597,
        0.46836844, 0.38244212, 0.49992633, 0.50362885, 0.42272332,
        0.5194877 , 0.6383